# **🧬 EnzyVision-AI++ | Interactive Enzyme Analysis**
EnzyVision-AI++ is an advanced AI-powered enzyme analysis platform that automatically extracts comprehensive features from enzyme activity images, identifies patterns through clustering and autoencoder-based dimensionality reduction, and visualizes enzyme behavior over time using heatmaps, temporal radars, and similarity graphs. It provides unsupervised feature importance insights, multi-batch comparisons, and inter-cluster analyses, while generating interactive 3D latent space visualizations and detailed PDF reports, enabling researchers to efficiently explore enzyme activity, detect correlations, and document experimental results in a highly intuitive and reproducible manner.

In [ ]:
# ============================================================
# 🧬 EnzyVision-AI++ | Full Combined Pipeline
# ============================================================

! pip install reportlab
import cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, networkx as nx
import plotly.express as px, plotly.graph_objects as go, torch, torch.nn as nn, torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import entropy
from scipy.spatial.distance import cdist
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import gradio as gr
import tempfile
import os

# ============================================================
# 1️⃣ FEATURE EXTRACTION (59+ Features)
# ============================================================
def extract_features(img):
    features = {}
    img = cv2.resize(img,(256,256))
    # Intensity
    features['mean_intensity'] = np.mean(img)
    features['std_intensity'] = np.std(img)
    features['variance'] = np.var(img)
    features['min_intensity'] = np.min(img)
    features['max_intensity'] = np.max(img)
    features['median_intensity'] = np.median(img)
    features['skewness'] = pd.Series(img.flatten()).skew()
    features['kurtosis'] = pd.Series(img.flatten()).kurtosis()
    features['energy'] = np.sum(img**2)
    features['entropy'] = entropy(np.histogram(img,bins=256)[0]+1)
    # Gradient
    gx = cv2.Sobel(img,cv2.CV_64F,1,0)
    gy = cv2.Sobel(img,cv2.CV_64F,0,1)
    grad = np.sqrt(gx**2 + gy**2)
    features['grad_mean'] = np.mean(grad)
    features['grad_std'] = np.std(grad)
    features['grad_max'] = np.max(grad)
    features['grad_energy'] = np.sum(grad**2)
    features['edge_density'] = np.sum(grad>50)/grad.size
    features['grad_entropy'] = entropy(np.histogram(grad,bins=256)[0]+1)
    features['grad_skew'] = pd.Series(grad.flatten()).skew()
    features['grad_kurtosis'] = pd.Series(grad.flatten()).kurtosis()
    # Morphology
    _,thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    perimeters = [cv2.arcLength(c,True) for c in contours]
    features['num_regions'] = len(contours)
    features['total_area'] = np.sum(areas)
    features['mean_area'] = np.mean(areas) if areas else 0
    features['max_area'] = np.max(areas) if areas else 0
    features['total_perimeter'] = np.sum(perimeters)
    features['mean_perimeter'] = np.mean(perimeters) if perimeters else 0
    features['compactness'] = (features['total_perimeter']**2)/(features['total_area']+1)
    features['fill_ratio'] = features['total_area']/img.size
    # FFT
    fft = np.fft.fftshift(np.fft.fft2(img))
    mag = np.abs(fft)
    features['fft_mean'] = np.mean(mag)
    features['fft_std'] = np.std(mag)
    features['fft_energy'] = np.sum(mag**2)
    features['fft_entropy'] = entropy(np.histogram(mag,bins=256)[0]+1)
    features['fft_low_freq'] = np.mean(mag[:50,:50])
    features['fft_high_freq'] = np.mean(mag[-50:,-50:])
    # Reaction Stability
    features['activity_density'] = np.sum(img>200)/img.size
    features['saturation_index'] = np.sum(img>230)/img.size
    features['reaction_uniformity'] = 1/(features['std_intensity']+1)
    features['activity_peak_ratio'] = features['max_intensity']/(features['mean_intensity']+1)
    features['spatial_dispersion'] = np.std(np.where(img>150)[0]) if np.any(img>150) else 0
    return features

# ============================================================
# 2️⃣ AUTOENCODER
# ============================================================
class AutoEncoder(nn.Module):
    def __init__(self,n):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n,64), nn.ReLU(),
            nn.Linear(64,16), nn.ReLU(),
            nn.Linear(16,3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3,16), nn.ReLU(),
            nn.Linear(16,64), nn.ReLU(),
            nn.Linear(64,n)
        )
    def forward(self,x):
        z = self.encoder(x)
        return self.decoder(z), z

# ============================================================
# 3️⃣ TEMPORAL RADAR + SIMILARITY GRAPH
# ============================================================
def generate_temporal_radar(df, steps=6):
    cluster_means = df.groupby('cluster').mean()
    categories = list(df.columns[:-1])
    frames=[]
    for t in range(steps):
        frame_data=[]
        for c in cluster_means.index:
            vals = cluster_means.loc[c].values*(1-0.05*t)
            frame_data.append(go.Scatterpolar(r=vals,theta=categories,fill='toself',name=f'Cluster {c}'))
        frames.append(go.Frame(data=frame_data,name=f'T{t}'))
    fig = go.Figure(
        data=[go.Scatterpolar(r=cluster_means.loc[c].values,theta=categories,fill='toself',name=f'Cluster {c}') for c in cluster_means.index],
        frames=frames
    )
    fig.update_layout(title='Temporal Radar Cluster Evolution',polar=dict(radialaxis=dict(visible=True)))
    return fig

def build_similarity_graph(latent):
    sim = cosine_similarity(latent)
    G = nx.Graph()
    for i in range(len(sim)):
        for j in range(i+1,len(sim)):
            if sim[i,j]>0.85:
                G.add_edge(i,j,weight=sim[i,j])
    return G

# ============================================================
# 4️⃣ TEMPORAL ENZYME ACTIVITY MAPS
# ============================================================
def generate_temporal_activity(base_map, steps=8):
    temporal_maps = []
    for t in range(steps):
        decay = np.exp(-0.15 * t)
        noise = np.random.normal(0,8, base_map.shape)
        img = np.clip(base_map*decay + noise,0,255)
        temporal_maps.append(img.astype(np.uint8))
    return temporal_maps

# ============================================================
# 5️⃣ MAIN PIPELINE + PDF GENERATION
# ============================================================
def run_pipeline_gradio(image):
    img = np.array(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = extract_features(img_gray)
    df = pd.DataFrame([features])

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df)

    # Clustering
    kmeans = KMeans(n_clusters=min(len(X_scaled),2),random_state=0,n_init='auto')
    df['cluster'] = kmeans.fit_predict(X_scaled)

    # Autoencoder
    model = AutoEncoder(X_scaled.shape[1])
    opt = optim.Adam(model.parameters(), lr=0.01)
    X_t = torch.tensor(X_scaled,dtype=torch.float32)
    for _ in range(300):
        recon, z = model(X_t)
        loss = ((recon-X_t)**2).mean()
        opt.zero_grad(); loss.backward(); opt.step()
    latent = z.detach().numpy()

    # Plots
    fig_heat = px.imshow(df.T, aspect='auto', title="Feature Heatmap (All Features)")
    fig_3d = px.scatter_3d(x=latent[:,0], y=latent[:,1], z=latent[:,2],
                           color=df.cluster.astype(str), title="Autoencoder Latent Space")
    fig_radar = generate_temporal_radar(df)

    # Similarity graph
    G = build_similarity_graph(latent)
    sim_path = tempfile.mktemp(".png")
    plt.figure(figsize=(6,6))
    pos = nx.spring_layout(G)
    nx.draw(G,pos,with_labels=True,node_color='skyblue',edge_color='gray',node_size=600)
    plt.title("Enzyme Similarity Graph")
    plt.savefig(sim_path)
    plt.close()

    # Feature & Cluster Heatmaps
    plt.figure(figsize=(14,10))
    corr = df.drop(columns=['cluster']).corr()
    sns.heatmap(corr, cmap="coolwarm", center=0, linewidths=0.1)
    plt.title("Global Feature Interaction Heatmap")
    plt.show()

    cluster_means = df.groupby("cluster").mean()
    plt.figure(figsize=(12,6))
    sns.heatmap(cluster_means, cmap="viridis", cbar=True)
    plt.title("Cluster-wise Enzyme Behavior Heatmap")
    plt.show()

    # PCA-based feature importance
    pca_full = PCA()
    pca_full.fit(X_scaled)
    importance = np.abs(pca_full.components_[0])
    feature_importance = pd.Series(importance, index=df.columns[:-1])
    top_features = feature_importance.sort_values(ascending=False)[:15]
    plt.figure(figsize=(8,4))
    top_features.plot(kind="barh")
    plt.title("Top Contributors to Enzyme Pattern Separation")
    plt.show()

    # PDF
    rules = [f"Cluster {c}:\n" + "\n".join([f"  {f} = {df[df.cluster==c][f].values[0]:.3f}" for f in df.columns[:-1]]) for c in df.cluster.unique()]
    pdf_path = "Enzyme_Report.pdf"
    doc = SimpleDocTemplate(pdf_path)
    styles = getSampleStyleSheet()
    content = [Paragraph("<b>🧬 EnzyVision-AI++ | Automated Enzyme Analysis</b>",styles["Title"]), Spacer(1,12)]
    for r in rules:
        content.append(Paragraph(r,styles["Normal"]))
        content.append(Spacer(1,6))
    content.append(Image(sim_path, width=400, height=400))
    doc.build(content)

    return fig_3d, fig_heat, fig_radar, "\n\n".join(rules), pdf_path

# ============================================================
# 6️⃣ GRADIO APP
# ============================================================
ui = gr.Interface(
    fn=run_pipeline_gradio,
    inputs=gr.Image(type='pil', label="Upload Enzyme Activity Image"),
    outputs=[
        gr.Plot(label="3D Latent Space"),
        gr.Plot(label="Feature Heatmap"),
        gr.Plot(label="Temporal Radar per Cluster"),
        gr.Textbox(label="All Feature Values per Cluster"),
        gr.File(label="PDF Report")
    ],
    title="🧬 EnzyVision-AI++ | Interactive Enzyme Analysis",
    description="Upload enzyme image → feature extraction, clustering, autoencoder, temporal radar, similarity graph, PDF report"
)

ui.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://428d75a48614cd65ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
